# Definindo um modelo de classificação com machine learning para analisar churn de clientes

## Objetivo do projeto

Este projeto tem como objetivo definir um modelo preditivo de classificação utilizando machine learning para avaliar os caso de churn de clientes em uma empresa.

Isto será feito de tal forma a avaliar as métricas de performance para cada modelo testado, escolhendo apenas um deles que apresentou o melhor desempenho. 

## Bibliotecas utilizadas